## Install libraries

In [71]:
!pip install -r requirements.txt

  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached numpy-2.1.3-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached scikit_learn-1.5.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached openai-1.55.0-py3-none-any.whl.metadata (24 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached tqdm-4.67.0-py3-none-any.whl.metadata (57 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached anyio-4.6.2.post1-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.7.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached pydantic-2.10.1-py3-none-any.whl.metadata (169 kB)
  Using cached sniffio-1.3.1-

## Create dummy graph data

In [72]:
import networkx as nx
import json

# Save Graph G1 to JSON
def save_graph_to_json(graph, filename):
    data = nx.readwrite.json_graph.node_link_data(graph)
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

# Create Graph G1 (Original Instruction Graph)
G1 = nx.DiGraph()

# Add nodes with unique titles, types, and descriptions
G1.add_node("Start", type="start", description="Inspect the electric vehicle for issues.")
G1.add_node("Battery Check", type="multiple", description="Is the battery charged?")
G1.add_node("Charge Battery", type="process", description="Charge the vehicle's battery.")
G1.add_node("Electrical System Check", type="process", description="Inspect the electrical connections and fuses.")
G1.add_node("Contact Technician", type="end", description="Call a certified EV technician.")
G1.add_node("Check Lights", type="process", description="Verify that all lights are functioning correctly.")  # New Node

# Add edges with yes/no decisions
G1.add_edge("Start", "Battery Check", answer="yes")
G1.add_edge("Battery Check", "Charge Battery", answer="no")
G1.add_edge("Battery Check", "Electrical System Check", answer="yes")
G1.add_edge("Electrical System Check", "Contact Technician", answer="no")
G1.add_edge("Electrical System Check", "Start", answer="yes")  # Loop for when all is fine
G1.add_edge("Start", "Check Lights", answer="no")  # New Edge

# Save Graph G1 to JSON
save_graph_to_json(G1, 'original_graph.json')

# Create Graph G2 (Modified Instruction Graph)
G2 = nx.DiGraph()

# Add nodes with unique titles, types, and descriptions
G2.add_node("Start", type="start", description="Inspect the vehicle for potential issues.")
G2.add_node("Battery Check", type="multiple", description="Check if the vehicle's battery is fully charged.")
G2.add_node("Replace Battery", type="process", description="Replace the battery if it cannot hold a charge.")
G2.add_node("Electrical System Check", type="process", description="Test the fuses and electrical wiring.")
G2.add_node("Contact Technician", type="end", description="Reach out to a professional technician for repairs.")
G2.add_node("Check Motor", type="process", description="Inspect the motor for unusual noise or damage.")  # New Node
G2.add_node("Check Lights", type="process", description="Ensure headlights and tail lights are operational.")  # Modified Node

# Add edges with yes/no decisions
G2.add_edge("Start", "Battery Check", answer="yes")
G2.add_edge("Battery Check", "Replace Battery", answer="no")  # Changed from "Charge Battery"
G2.add_edge("Battery Check", "Electrical System Check", answer="yes")
G2.add_edge("Electrical System Check", "Contact Technician", answer="no")
G2.add_edge("Electrical System Check", "Check Motor", answer="yes")  # New Edge
G2.add_edge("Check Motor", "Start", answer="yes")  # New loop
G2.add_edge("Start", "Check Lights", answer="no")  # Retained Edge

# Save Graph G2 to JSON
save_graph_to_json(G2, 'curated_graph.json')

print("Graphs saved to 'original_graph.json' and 'curated_graph.json'.")


Graphs saved to 'original_graph.json' and 'curated_graph.json'.


/Users/akihiroinui/Projects/unnamed_project/.venv/lib/python3.12/site-packages/networkx/readwrite/json_graph/node_link.py:142: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(


## Load the Graph from JSON

In [74]:
# Function to load graph from JSON
def load_graph_from_json(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
    return nx.readwrite.json_graph.node_link_graph(data)

# Load instruction graphs
original_graph = load_graph_from_json('original_graph.json')
curated_graph = load_graph_from_json('curated_graph.json')

## Text based edit distance calculation

In [75]:
def compute_edit_distance(source_graph:nx.Graph, target_graph: nx.Graph) -> float:
    """
    Compute the Graph Edit Distance (GED) between two graphs.
    Args:
        source_graph: Source NetworkX graph
        target_graph: Target NetworkX graph
    Returns:
        edit_distance: Graph Edit Distance between the two graphs
    """
    # Here, you can define the node match criteria. It checks the type AND description attributes of the nodes.
    node_match_criteria = lambda source_node, target_node: (
        source_node.get("type") == target_node.get("type") and source_node.get("description") == target_node.get("description")
    )
    # Here, you can define the edge match criteria. It checks the answer attribute of the edges.
    edge_match_criteria = lambda source_edge, target_edge: source_edge.get("answer") == target_edge.get("answer")

    edit_distance = nx.graph_edit_distance(source_graph, target_graph, node_match=node_match_criteria, edge_match=edge_match_criteria)
    return edit_distance

In [76]:
graph_edit_distance = compute_edit_distance(original_graph, curated_graph)
print(f"Text based Graph Edit Distance (using node type and description): {int(graph_edit_distance)}")

Text based Graph Edit Distance (using node type and description): 10


## Use node semantic similarity to calculate edit distance

In [77]:
import os
import numpy as np
import networkx as nx
from dotenv import load_dotenv
from openai import AzureOpenAI
from sklearn.metrics.pairwise import cosine_similarity

# Prepare your AOAI environment
load_dotenv()
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version = "2023-05-15",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

def generate_embedding(text: str, model="text-embedding-3-large") -> np.ndarray:
    """
    Generate embedding for a given text using the specified model.

    Args:
        text (str): input text to generate embedding for
        model (str, optional): model to use for generating embedding. Defaults to "text-embedding-3-large".

    Returns:
        embedding: Generated embedding for the input text
    """
    return client.embeddings.create(input = [text], model=model).data[0].embedding


# Compute cosine similarity between two embeddings
def compute_cosine_similarity(source_embedding: np.ndarray, target_embedding: np.ndarray) -> float:
    """
    Calculate the cosine similarity between two embeddings.

    Args:
        source_embedding (np.ndarray): Source embedding
        target_embedding (np.ndarray): Target embedding

    Returns:
        similarity: Cosine similarity between the two embeddings
    """
    return cosine_similarity([source_embedding], [target_embedding])[0][0]


def add_embeddings_to_graph(graph: nx.Graph) -> nx.Graph:
    """
    Generate embeddings for the node descriptions in the graph and add them to the graph.

    Args:
        graph (_type_): _description_
    """
    for node_id in graph.nodes:
        description = graph.nodes[node_id]["description"]
        embedding = generate_embedding(description)
        graph.nodes[node_id]["embedding"] = embedding
    return graph


# Load graphs
original_graph = load_graph_from_json('original_graph.json')
curated_graph = load_graph_from_json('curated_graph.json')

# Add embeddings to the graphs
add_embeddings_to_graph(original_graph)
add_embeddings_to_graph(curated_graph)
print("Added embeddings to the graphs.")

Added embeddings to the graphs.


In [78]:
# Compute Graph Edit Distance with semantic similarity
def compute_semantic_edit_distance(source_graph: nx.Graph, traget_graph: nx.Graph, threshold=0.8) -> float:
    """
    Compute the Graph Edit Distance (GED) between two graphs based on semantic similarity on node descriptions.

    Args:
        source_graph (nx.Graph): Source graph
        traget_graph (nx.Graph): Target graph
        threshold (float, optional): Threshold to determine the nodes as similar ones. Defaults to 0.8.

    Returns:
        graph_edit_disntance: Graph Edit Distance between the two graphs
    """
    # Node match function based on semantic similarity
    def node_match_criteria(source_node, target_node):
        source_node_embedding = source_node.get("embedding")
        target_node_embedding = target_node.get("embedding")
        if source_node_embedding is not None and target_node_embedding is not None:
            similarity = compute_cosine_similarity(source_node_embedding, target_node_embedding)
            if similarity >= threshold:
                # For debugging
                print(f"""
                      Similarity between 
                      '{source_node['description']}' 
                      and 
                      '{target_node['description']}'
                      is {np.round(similarity, 3)} and above threshold {threshold}")
                      """)
                return True
        return False

    # Edge match function based on exact 'answer' attribute
    def edge_match_criteria(source_edge, target_edge):
        return source_edge.get("answer") == target_edge.get("answer")

    # Compute graph edit distance
    graph_edit_disntance = nx.graph_edit_distance(
        source_graph, traget_graph, node_match=node_match_criteria, edge_match=edge_match_criteria
    )
    return graph_edit_disntance

# Compute the semantic graph edit distance
threshold = 0.8
semantic_ged = compute_semantic_edit_distance(original_graph, curated_graph, threshold)
print(f"Semantic Graph Edit Distance (threshold={threshold}): {int(semantic_ged)}")


                      Similarity between 
                      'Inspect the electric vehicle for issues.' 
                      and 
                      'Inspect the vehicle for potential issues.'
                      is 0.803 and above threshold 0.8")
                      

                      Similarity between 
                      'Inspect the electrical connections and fuses.' 
                      and 
                      'Test the fuses and electrical wiring.'
                      is 0.808 and above threshold 0.8")
                      
Semantic Graph Edit Distance (threshold=0.8): 8
